In [8]:
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import ZebraLib as zb
from uncertainties import ufloat
from uncertainties.umath import *  # sin(), etc.

#Viscosidade e densidade da água Pa.s, kg/m^3
mi = 1.002e-3
rho = 1

#Dados diâmetros - Incerteza tipo A
D_val = np.array([31.5, 30.8, 29.5, 29.0, 31.6, 32.1, 31.0, 31.7, 31.5, 32])/1000 #m
D = ufloat(D_val.mean(), D_val.std(ddof=1))

#Dados tempo e volumes - Incerteza tipo B
t_val = np.array([43.68, 12.36, 18.56, 20.29, 6.73, 3.30])    #s
V_val = np.array([0.490, 0.495, 1, 0.985, 0.990, 0.820])/1000 #m^3
t, V = [], []
for i in range(len(t_val)):
    t.append(ufloat(t_val[i], 0.01))
    V.append(ufloat(V_val[i], 0.01/1000)) #Convertendo o erro p/ m^3

In [9]:
print(f't:{t}') #tempo(s)
print(f'V:{V}') #Volume m^3

t:[43.68+/-0.01, 12.36+/-0.01, 18.56+/-0.01, 20.29+/-0.01, 6.73+/-0.01, 3.3+/-0.01]
V:[0.00049+/-1e-05, 0.000495+/-1e-05, 0.001+/-1e-05, 0.000985+/-1e-05, 0.00099+/-1e-05, 0.00082+/-1e-05]


In [10]:
V_bar, Re = [], []
for i in range(len(t_val)):
    NUM = ((V[i])/t[i]) #multipliquei por x1000 pra ficar em ml
    DEM = ((D/2)**2)*np.pi
    V_bar.append(NUM/DEM)
    Re.append(((NUM/DEM)*D*rho)/(mi))

In [11]:
print(f'Velocidade:{V_bar}') # m/s
print(f'Re:{Re}') #numero

Velocidade:[0.01479588892892419+/-0.0010387643906070833, 0.05282193911657541+/-0.003705508188293083, 0.07106399855025167+/-0.004826426531598903, 0.06402974844239925+/-0.004350108093632587, 0.19402055497202733+/-0.013182858109921305, 0.3277385474957546+/-0.022397472514358183]
Re:[0.45879068764638176+/-0.018031348147128076, 1.6379018446626723+/-0.06421056594614846, 2.2035513322917355+/-0.07723076534982588, 1.9854334172708028+/-0.06967162816025207, 6.0161862704400075+/-0.21119577256610708, 10.162511647398297+/-0.36443593614151204]


In [19]:
df = pd.DataFrame({'D(m)':D, 't(s)':t, 'V(m^3)':V, 'V_bar(m/s)':V_bar, 'Re':Re})
#df.to_excel('dados_1.xlsx')
#df[['t(s)','V(m^3)']]
df[['V_bar(m/s)','Re']]

,V_bar(m/s),Re
0,0.0148+/-0.0010,0.459+/-0.018
1,0.053+/-0.004,1.64+/-0.06
2,0.071+/-0.005,2.20+/-0.08
3,0.064+/-0.004,1.99+/-0.07
4,0.194+/-0.013,6.02+/-0.21
5,0.328+/-0.022,10.2+/-0.4
